In [1]:
import os, math
from PIL import Image, ImageDraw, ImageFont
from tensorflow.keras.datasets import cifar10, mnist
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, BatchNormalization 
from tensorflow.keras.layers import Conv2D, Flatten, Activation, Concatenate
from tensorflow.keras.layers import Reshape, Conv2DTranspose, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import mse, SparseCategoricalCrossentropy


In [2]:
tf.random.set_seed(42)

In [3]:
(x_train, y_train),(x_test, _) = mnist.load_data()

In [4]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_train = x_train / 255

In [5]:
x_train.shape

(60000, 28, 28, 1)

In [6]:
image_size = x_train[0].shape[1]
image_size

28

In [7]:
class ConvTransBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides):
        super().__init__()
        self.bn = BatchNormalization()  # check training flag
        self.act = Activation(activation='relu')
        self.conv2D_trans = Conv2DTranspose(filters=filters,
                                kernel_size=kernel_size,
                                strides=strides,
                                padding='same')
        
    def call(self, inputs, training=False):
        x = self.bn(inputs)
        x = self.act(x)
        return self.conv2D_trans(x)


In [8]:
class Generator(tf.keras.models.Model):
    def __init__(self, filters, kernel_size, resize_img):
        super().__init__()
        self.dense1 = Dense(resize_img * resize_img * filters[0]) # 28x28x128
        self.reshape = Reshape([resize_img, resize_img, filters[0]])
        self.conv2dtrans = []
        for i, _filter in enumerate(filters):
            if i <= 1:
                strides = 2
            else:
                strides = 1
            self.conv2dtrans.append(ConvTransBlock(_filter, kernel_size, strides))
        
        self.act = Activation("sigmoid")

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        x = self.reshape(x)
        for conv in self.conv2dtrans:
            x = conv(x)
        return self.act(x)
    
    def model(self):
        x = Input(shape=[1, 6272])
        return Model(inputs=x, outputs=self.call(x))

In [9]:
def leaky_conv(filters, kernel_size, strides):
    return Sequential([LeakyReLU(alpha=0.2),
                       Conv2D(filters=filters,
                       kernel_size=kernel_size,
                       strides=strides,
                       padding='same')])

In [10]:
# inp1 = Input(shape=[28,28,1])
# inp2 = Input(shape=[10,])
# input_dense = Dense(28*28)(inp2)
# reshaped = Reshape((28,28,1))(input_dense)
# concat = Concatenate()([inp1, reshaped])
# conv = Conv2D(32, 3, 1, "same")(concat)
# conv = Flatten()(conv)
# out = Dense(10)(conv)
# model = Model(inputs=(inp1, inp2), outputs=out)
         
model.compile(loss="categorical_crossentropy")

model.fit((x_train, one_hot), one_hot) 

NameError: ignored

In [11]:
class Discriminator(tf.keras.models.Model):
    def __init__(self, filters, kernel_size):
        super().__init__()
        self.leaky_convs = []
        for i, _filter in enumerate(filters):
            if i < len(filters) - 1:
                strides = 2
            else:
                strides = 1
            self.leaky_convs.append(leaky_conv(_filter, kernel_size, strides))
        self.flat = Flatten()
        self.dense = Dense(1, activation='sigmoid')
        self.input_dense = Dense(28*28)
        self.reshaped = Reshape((28,28,1))
        self.contact = Concatenate()

    def call(self, inputs, training=False):
        img, label = inputs   
        print("tu działa")
        huge_label = self.input_dense(label)
        print("tu być może")    
        huge_reshaped_label = self.reshaped(huge_label)
        print("huge_reshaped_label", huge_reshaped_label)
        print()       
        x = self.contact([img, huge_reshaped_label])  # tensor 3 dim (28, 28 ,2)   
        print("jak tu się uda to hurra!!!")
        print("x", x) 
        for conv in self.leaky_convs:
            x = conv(x)
        x = self.flat(x)
        return self.dense(x)

In [16]:
disc = Discriminator(disc_layers_filters, disc_kernel_size)
disc.compile(loss='binary_crossentropy',
             optimizer=RMSprop(learning_rate=disc_lr, decay=disc_decay),
             metrics=['accuracy'])

In [17]:
inp1 = Input(shape=[28,28,1])
inp2 = Input(shape=[10,])
x = disc([inp1, inp2])


tu działa
tu być może
huge_reshaped_label Tensor("discriminator/reshape/Reshape:0", shape=(None, 28, 28, 1), dtype=float32)

jak tu się uda to hurra!!!
x Tensor("discriminator/concatenate/concat:0", shape=(None, 28, 28, 2), dtype=float32)


In [20]:
labels = np.ones(shape=x_train.shape[0])
disc.fit([x_train, one_hot], labels)

tu działa
tu być może
huge_reshaped_label Tensor("discriminator/reshape/Reshape:0", shape=(32, 28, 28, 1), dtype=float32)

jak tu się uda to hurra!!!
x Tensor("discriminator/concatenate/concat:0", shape=(32, 28, 28, 2), dtype=float32)
tu działa
tu być może
huge_reshaped_label Tensor("discriminator/reshape/Reshape:0", shape=(32, 28, 28, 1), dtype=float32)

jak tu się uda to hurra!!!
x Tensor("discriminator/concatenate/concat:0", shape=(32, 28, 28, 2), dtype=float32)
 177/1875 [=>............................] - ETA: 2:23 - loss: 0.0118 - accuracy: 0.9979

KeyboardInterrupt: ignored

In [25]:
#hiper params
latent_size = 2
batch_size = 64
train_steps = 40000
disc_lr = 2e-4
disc_decay = 6e-8
gen_lr = disc_lr/4
gen_decay = disc_decay/4

In [26]:
disc_kernel_size = 5
disc_layers_filters = [32, 64, 128]

In [27]:
gen_resize_img = image_size // 4
gen_kernel_size = 3
gen_layers_filter = [128, 64, 32, 1]

In [28]:
gen = Generator(gen_layers_filter, gen_kernel_size, gen_resize_img)

In [34]:
class Gan(tf.keras.models.Model):
    def __init__(self):
        super().__init__()
        self.gen = Generator(gen_layers_filter, gen_kernel_size, gen_resize_img)
        self.disc = Discriminator(disc_layers_filters, disc_kernel_size)
        # self.disc.compile(loss='binary_crossentropy',
        #      optimizer=RMSprop(learning_rate=disc_lr, decay=disc_decay),
        #      metrics=['accuracy'])


    def call(self, inputs):
        noise, label = inputs
        fake_img = self.gen(noise)
        return self.disc((fake_img, label))
    
    def model(self):
        x = Input(shape=[1, 6272])
        return Model(inputs=x, outputs=self.call(x))
        

gan = Gan()
gan.compile(loss='binary_crossentropy',
                optimizer=RMSprop(learning_rate=gen_lr, decay=gen_decay),
                metrics=['accuracy'])     



        

# def build_gan():
#     gen_input = Input(shape=(latent_size + 10,))   
#     # fake_img = Generator(gen_layers_filter, gen_kernel_size, gen_resize_img)(gen_input)
#     # disc_input = Input((10,))
#     # output = disc((fake_img, disc_input))
#     gan = Model(inputs=gen_input, outputs=output)  
#     gan.summary()
#     gan.compile(loss='binary_crossentropy',
#                 optimizer=RMSprop(learning_rate=gen_lr, decay=gen_decay),
#                 metrics=['accuracy'])            
#     return gan   


# gan = build_gan()

In [22]:
noise = np.random.uniform(-1, 1, size=(x_train.shape[0], latent_size))
noise.shape

(60000, 2)

In [23]:
one_hot = tf.one_hot(y_train, y_train.max()+1)
one_hot.shape

TensorShape([60000, 10])

In [24]:
fake_labels = np.zeros(y_train.shape[0])
fake_labels.shape

(60000,)

In [30]:
noise.shape, one_hot.shape, fake_labels.shape, x_train.shape

((60000, 2), TensorShape([60000, 10]), (60000,), (60000, 28, 28, 1))

In [35]:
gan.fit((noise, one_hot), labels, batch_size=32)

tu działa
tu być może
huge_reshaped_label Tensor("gan_2/discriminator_3/reshape_7/Reshape:0", shape=(32, 28, 28, 1), dtype=float32)

jak tu się uda to hurra!!!
x Tensor("gan_2/discriminator_3/concatenate_3/concat:0", shape=(32, 28, 28, 2), dtype=float32)
tu działa
tu być może
huge_reshaped_label Tensor("gan_2/discriminator_3/reshape_7/Reshape:0", shape=(32, 28, 28, 1), dtype=float32)

jak tu się uda to hurra!!!
x Tensor("gan_2/discriminator_3/concatenate_3/concat:0", shape=(32, 28, 28, 2), dtype=float32)
  31/1875 [..............................] - ETA: 10:29 - loss: 0.2244 - accuracy: 0.9889

KeyboardInterrupt: ignored

In [ ]:
gan.model().summary()


In [ ]:
X = tf.random.uniform((1,6272))
for i, layer in enumerate(gan.layers):
    print(i, layer.__class__.__name__)
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

In [ ]:
for i, layer in enumerate(gan.layers):
    

In [ ]:
gen.model().summary()
